In [1]:
import requests, json, time
import pandas as pd
from collections import Counter, defaultdict

In [2]:
do_read_from_web = True
do_read_from_cvs = not do_read_from_web

In [ ]:
base_url = "https://inspirehep.net/api/literature"

# Query: papers affiliated with Stony Brook from 2022 to now
query = 'aff:"SUNY, Stony Brook" and date 2022->2025'
query = 'aff:"Stony Brook U." and date 2022->2025'
query = '(aff:"Stony Brook U." or aff:"SUNY, Stony Brook" or aff:"SUNY" or aff:"YITP") and date 2022->2025'


exclude_collabs = {"ATLAS", "CMS"}

params = {
    "q": query,
    "size": 25,
    "sort": "mostrecent"
}

records = []
page = 1

In [4]:
if do_read_from_web:
    while True:
        params["page"] = page
        r = requests.get(base_url, params=params)
        data = r.json()
        hits = data.get("hits", {}).get("hits", [])
        if not hits:
            break

        for hit in hits:
            m = hit.get("metadata", {})

            # collaborations
            collabs = [c.get("value", "") for c in m.get("collaborations", [])]
            # skip excluded large collaborations
            if any(c in exclude_collabs for c in collabs):
                continue

            # --- Basic metadata ---
            rec = {
                "Title": m.get("titles", [{}])[0].get("title", ""),
                "Date": m.get("preprint_date") or m.get("publication_info", [{}])[0].get("year"),
                "DOI": m.get("dois", [{}])[0].get("value", ""),
                "Collaboration": ", ".join(c for c in collabs if c),
                "Citations": m.get("citation_count", 0)
            }

            # --- Detailed author info (names + affiliations) ---
            author_list = []
            for a in m.get("authors", []):
                # merge both "affiliations" and "raw_affiliations"
                affs = [
                    aff.get("value", "")
                    for aff in (a.get("affiliations", []) + a.get("raw_affiliations", []))
                    if "value" in aff
                ]
                author_list.append({
                    "name": a.get("full_name", ""),
                    "affiliations": affs
                })
            rec["Authors"] = author_list

            records.append(rec)

        print(f"Fetched page {page} with {len(hits)} records...")
        page += 1
        time.sleep(0.5)  # be polite to API

    print(f"Total records: {len(records)}")


Fetched page 1 with 100 records...
Fetched page 2 with 100 records...
Fetched page 3 with 100 records...
Fetched page 4 with 100 records...
Fetched page 5 with 100 records...
Fetched page 6 with 100 records...
Fetched page 7 with 100 records...
Fetched page 8 with 100 records...
Fetched page 9 with 100 records...
Fetched page 10 with 100 records...
Fetched page 11 with 100 records...
Fetched page 12 with 100 records...
Fetched page 13 with 100 records...
Fetched page 14 with 100 records...
Fetched page 15 with 100 records...
Fetched page 16 with 100 records...
Fetched page 17 with 100 records...
Fetched page 18 with 49 records...
Total records: 1332


In [5]:
import requests, json

r = requests.get("https://inspirehep.net/api/literature/2640016")
m = r.json()["metadata"]

# print the entire metadata nicely formatted
print(json.dumps(m, indent=2))


{
  "publication_info": [
    {
      "year": 2023,
      "artid": "C06024",
      "journal_issue": "06",
      "journal_title": "JINST",
      "journal_record": {
        "$ref": "https://inspirehep.net/api/journals/1212215"
      },
      "journal_volume": "18"
    }
  ],
  "citation_count_without_self_citations": 0,
  "citation_count": 0,
  "dois": [
    {
      "value": "10.1088/1748-0221/18/06/C06024",
      "source": "IOP"
    },
    {
      "value": "10.1088/1748-0221/18/06/C06024",
      "source": "arXiv",
      "material": "publication"
    }
  ],
  "titles": [
    {
      "title": "The effects of a passive Bi-Polar Grid (BPG) on Ion Back-Flow (IBF) and resolution",
      "source": "IOP"
    },
    {
      "title": "The effects of a passive Bi-Polar Grid (BPG) on Ion Back-Flow (IBF) and Resolution",
      "source": "arXiv"
    }
  ],
  "$schema": "https://inspirehep.net/schemas/records/hep.json",
  "authors": [
    {
      "uuid": "9e322b59-db96-4e6b-90cf-b91ae9c839c0",
      

In [6]:
if do_read_from_web: 
    
    with open("inspire_full_sbu_2022_2025.json", "w") as f:
        json.dump(records, f, indent=2)
    
    # Optionally also make a simplified DataFrame for quick viewing
    df = pd.DataFrame([{
        "Title": r["Title"],
        "Date": r["Date"],
        "Collaboration": r["Collaboration"],
        "Citations": r["Citations"],
        "NumAuthors": len(r["Authors"])
    } for r in records])
    
    df.to_csv("inspire_full_sbu_2022_2025.csv", index=False)
    df.head()

In [7]:
if do_read_from_cvs:
    df = pd.read_csv("inspire_sbu_2022_2025.csv")
    print(f"Total records from CSV: {len(df)}")
    df.head(10)
    with open("inspire_full_sbu_2022_2025.json") as f:
        records = json.load(f)

In [8]:
df["Collaboration"].value_counts()

Collaboration
                                                         897
STAR                                                      65
IceCube                                                   61
PHENIX                                                    33
DUNE                                                      29
                                                        ... 
LIGO Scientific, KAGRA, Virgo, Swift, Swift-BAT/GUANO      1
MOLLER                                                     1
HERA                                                       1
ATLAS Computing activity                                   1
LIGO Scientific, Virgo, VIRGO                              1
Name: count, Length: 108, dtype: int64

In [9]:
# Keep only records with collaboration containing 'PHENIX' or 'sPHENIX'
df_filtered = df[df['Collaboration'].str.contains(r'\b(s?PHENIX)\b', case=False, na=False)]

# Optional: reset index for cleanliness
df_filtered = df_filtered.reset_index(drop=True)

# Display
print(f"Filtered to {len(df_filtered)} PHENIX/sPHENIX records")
df_filtered.head(10)

Filtered to 37 PHENIX/sPHENIX records


/tmp/ipykernel_33721/486019506.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_filtered = df[df['Collaboration'].str.contains(r'\b(s?PHENIX)\b', case=False, na=False)]


,Title,Date,Collaboration,Citations,NumAuthors
0,Transverse single-spin asymmetry of forward $η...,2025-09-16,PHENIX,0,383
1,Cross sections of $\eta$ mesons in $p$ $+$ $p$...,2025-07-07,PHENIX,1,500
2,Low-mass vector-meson production at forward ra...,2025-07-06,PHENIX,0,355
3,Measurement of inclusive jet cross section and...,2025-06-15,PHENIX,1,447
4,Azimuthal anisotropy of direct photons in Au$+...,2025-04-03,PHENIX,2,325
5,Measurement of charged hadron multiplicity in ...,2025-04-02,sPHENIX,1,305
6,Measurement of the transverse energy density i...,2025-04-02,sPHENIX,0,305
7,Measurements at forward rapidity of elliptic f...,2024-09-19,PHENIX,0,370
8,Measurement of elliptic flow of <math><mrow><m...,2024-09-19,PHENIX,2,370
9,"Multiplicity dependent <math display=""inline"">...",2024-09-05,PHENIX,6,303


In [10]:
# Keywords that identify SBU affiliations
sbu_keys = ["stony brook", "suny stony brook", "state univ. of new york"]

In [11]:
author_counter = Counter()
author_papers = {}  # optional: to track which papers belong to each author

for rec in records:
    title = rec["Title"]
    for author in rec["Authors"]:
        affs = author.get("affiliations", [])
        # Check if any affiliation matches SBU
        if any(any(key in aff.lower() for key in sbu_keys) for aff in affs):
            name = author["name"]
            author_counter[name] += 1
            author_papers.setdefault(name, []).append(title)

print(f"Found {len(author_counter)} authors with SBU affiliation")

Found 773 authors with SBU affiliation


In [12]:
authors_df = pd.DataFrame(
    author_counter.items(),
    columns=["Author", "Num_SBU_Papers"]
).sort_values("Num_SBU_Papers", ascending=False)

authors_df.head(20)

,Author,Num_SBU_Papers
58,"Kiryluk, J.",84
57,"Chen, Z.",84
59,"Zhang, Z.",83
120,"Deshpande, A.",62
53,"Park, S.",55
314,"Hamdaoui, H.",55
2,"Zahed, Ismail",54
19,"Jung, C.K.",45
122,"Drees, A.",41
125,"Gal, C.",41


In [13]:
authors_full = pd.DataFrame([
    {"Author": a, 
     "Num_SBU_Papers": len(set(author_papers[a])),
     "Titles": "; ".join(sorted(set(author_papers[a])))}
    for a in author_papers
]).sort_values("Num_SBU_Papers", ascending=False)

authors_full.to_csv("inspire_authors_sbu_summary.csv", index=False)

In [14]:
import fitz  # PyMuPDF
import re
pdf_file_name = "/home/yoren/yumvd.Yandex.Disk/Yura/Personal/grants/DOE_progress_report_publications_2023_2026.pdf"

In [15]:
import fitz
import re

pdf_path = pdf_file_name


# read all text
doc = fitz.open(pdf_path)
text = " ".join(page.get_text("text") for page in doc)

# --- normalize all hyphen / dash variants ---
dash_chars = [
    "\u00ad",  # soft hyphen
    "\u2010",  # hyphen
    "\u2011",  # non-breaking hyphen
    "\u2012",  # figure dash
    "\u2013",  # en dash
    "\u2014",  # em dash
    "\u2015",  # horizontal bar
    "\u2212",  # minus sign
]
for ch in dash_chars:
    text = text.replace(ch, "-")

# remove newlines inside DOIs
text = re.sub(r'\s*-\s*(?=\d)', '-', text)  # fix broken numeric segments
text = re.sub(r'\s*\n\s*', ' ', text)        # flatten newlines

# --- extract DOIs ---
pattern = r'10\.\d{4,9}/[A-Za-z0-9.\-()/]+'
dois = re.findall(pattern, text)

dois = sorted(set(dois))
print(f"Found {len(dois)} DOIs:")
for d in dois:
    print(d)


Found 26 DOIs:
10.1007/JHEP08(2025)075
10.1016/j.nima.2024.170127
10.1016/j.nima.2024.170171
10.1016/j.physletb.2023.138101
10.1088/1748-0221/18/06/C06024
10.1103/6pmd-6dwr
10.1103/PhysRevC.107.014907
10.1103/PhysRevC.107.024907
10.1103/PhysRevC.107.024914
10.1103/PhysRevC.109.044912
10.1103/PhysRevC.109.054910
10.1103/PhysRevC.110.044901
10.1103/PhysRevC.110.064905
10.1103/PhysRevC.110.064909
10.1103/PhysRevD.107.052012
10.1103/PhysRevD.107.112004
10.1103/PhysRevD.108.072016
10.1103/PhysRevLett.130.251901
10.1103/PhysRevLett.134.022302
10.1103/h8d5-swg6
10.1103/hpm9-
10.1103/ptpm-jtt8
10.1140/epjc/s10052-024-12987-0
10.1140/epjc/s10052-024-13003-1
10.1140/epjc/s10052-024-13115-8
10.1140/epjc/s10052-024-13416-y


In [16]:
import re

# --- Helper: normalize DOIs to a consistent comparable form ---
def normalize_doi(d):
    """Lowercase, strip, and remove common prefixes."""
    if not d:
        return ""
    d = d.lower().strip()
    d = d.replace("https://doi.org/", "")
    d = d.replace("http://doi.org/", "")
    d = d.replace("doi:", "")
    d = re.sub(r"\s+", "", d)   # remove spaces and line breaks
    return d

# --- Normalize DOE list of DOIs ---
doe_dois = [normalize_doi(d) for d in dois]

# --- Define Stony Brook affiliation keywords ---
sbu_keys = [
    "stony brook",
    "suny stony brook",
    "state univ. of new york",
    "yitp",
    "yitp, stony brook",
    "stony brook u.",
    "suny, stony brook",
    "suny"
]

# --- Step 1: find all matched records by normalized DOI ---
matched_records = []
for r in records:
    rec_doi = normalize_doi(r.get("DOI", ""))
    if rec_doi and rec_doi in doe_dois:
        matched_records.append(r)

print(f"Matched {len(matched_records)} publications between DOE list and database.")

# --- Diagnostic: find unmatched DOIs (to catch hidden mismatches) ---
unmatched_dois = [
    d for d in doe_dois
    if not any(d == normalize_doi(r.get("DOI", "")) for r in records)
]
if unmatched_dois:
    print(f"\n⚠️  {len(unmatched_dois)} DOIs from DOE list not found exactly in records. Close matches:")
    for d in unmatched_dois:
        close = [
            r.get("DOI", "") for r in records
            if normalize_doi(d) in normalize_doi(r.get("DOI", "")) or normalize_doi(r.get("DOI", "")) in normalize_doi(d)
        ]
        print(" ", d, "→", close)
else:
    print("✅ All DOIs matched successfully.")

# --- Step 2: extract all authors with at least one SBU affiliation ---
sbu_authors = set()

for rec in matched_records:
    for a in rec.get("Authors", []):
        affs = a.get("affiliations", [])
        # If any affiliation string matches SBU keywords
        if any(any(k in aff.lower() for k in sbu_keys) for aff in affs):
            sbu_authors.add(a["name"])

print(f"\nFound {len(sbu_authors)} unique authors with SBU affiliation in matched publications.")
for name in sorted(sbu_authors):
    print("-", name)


Matched 23 publications between DOE list and database.

⚠️  3 DOIs from DOE list not found exactly in records. Close matches:
  10.1016/j.nima.2024.170171 → ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''

In [17]:
import re
from collections import defaultdict

def get_last_name(full_name):
    """Extract surname from 'Last, First' or 'First Last' formats."""
    if not full_name:
        return ""
    name = re.sub(r"[,]", " ", full_name).strip()
    parts = [p for p in name.split() if p]
    if not parts:
        return ""
    last = parts[0] if "," in full_name else parts[-1]
    last = re.sub(r"[^A-Za-z\-]", "", last).lower().strip()
    return last

def get_first_initial(full_name):
    """Return the first letter of the given name part (after comma or first token)."""
    if not full_name:
        return ""
    name = re.sub(r"[,]", " ", full_name).strip()
    parts = [p for p in name.split() if p]
    if not parts:
        return ""
    # if format 'Last, First'
    if "," in full_name:
        if len(parts) > 1:
            token = re.sub(r"[^A-Za-z]", "", parts[1])
            return token[0].lower() if token else ""
    # if 'First Last'
    elif len(parts) > 1:
        token = re.sub(r"[^A-Za-z]", "", parts[0])
        return token[0].lower() if token else ""
    return ""

# ------------------------------------------------------
# Combine authors: same last name + same first initial
# ------------------------------------------------------
author_groups = defaultdict(list)

for name in sorted(sbu_authors):
    last = get_last_name(name)
    first_init = get_first_initial(name)
    if last:
        key = f"{last}_{first_init}"  # e.g., "esha_r", "chen_c"
        author_groups[key].append(name)

# Show groups that contain multiple variants
for key, names in sorted(author_groups.items()):
    if len(names) > 1:
        print(f"{key}: {names}")

print(f"\nTotal unique (last+first-initial) combinations: {len(author_groups)}")


esha_r: ['Esha, R.', 'Esha, Roli']
hemmick_t: ['Hemmick, T.', 'Hemmick, T.K.']
jacak_b: ['Jacak, B.', 'Jacak, B.V.']

Total unique (last+first-initial) combinations: 101


In [18]:
# ----------------------------------------------------
# Build unified author keys (last + first initial)
# ----------------------------------------------------
author_variants = defaultdict(list)
for name in sorted(sbu_authors):
    last = get_last_name(name)
    first_init = get_first_initial(name)
    if not last:
        continue
    key = f"{last}_{first_init}"
    author_variants[key].append(name)

sbu_authors_last = set(author_variants.keys())
print(f"Unified to {len(sbu_authors_last)} unique (last+first-initial) authors for DOE-matched list.")

# ----------------------------------------------------
# Count 2023+ SBU-affiliated papers for those authors
# ----------------------------------------------------
def year_from_record(rec):
    y = str(rec.get("Date", ""))[:4]
    return int(y) if y.isdigit() else None

records_recent = [r for r in records if (year_from_record(r) and year_from_record(r) >= 2023)]
print(f"Total records from 2023+: {len(records_recent)}")

author_recent_sbu = defaultdict(list)

for rec in records_recent:
    doi = rec.get("DOI", "")
    for a in rec.get("Authors", []):
        last = get_last_name(a["name"])
        first_init = get_first_initial(a["name"])
        key = f"{last}_{first_init}"
        if key not in sbu_authors_last:
            continue
        affs = a.get("affiliations", [])
        if any(any(k.lower() in aff.lower() for k in sbu_keys) for aff in affs):
            author_recent_sbu[key].append(doi)

# ----------------------------------------------------
# Build DataFrame
# ----------------------------------------------------
authors_recent_df = pd.DataFrame([
    {
        "AuthorKey": key,
        "FullNames": ", ".join(sorted(set(author_variants.get(key, [])))),
        "Num_SBU_Papers_since2023": len(set(dois)),
        "DOIs": "; ".join(sorted(set(dois)))
    }
    for key, dois in author_recent_sbu.items()
]).sort_values("Num_SBU_Papers_since2023", ascending=False)

authors_recent_df.to_csv("authors_sbu_since2023.csv", index=False)

print(f"Saved {len(authors_recent_df)} authors with SBU affiliation since 2023.")
authors_recent_df.head(30)

Unified to 101 unique (last+first-initial) authors for DOE-matched list.
Total records from 2023+: 818
Saved 100 authors with SBU affiliation since 2023.


,AuthorKey,FullNames,Num_SBU_Papers_since2023,DOIs
12,deshpande_a,"Deshpande, A.",35,; 10.1007/s41781-024-00113-4; 10.1016/j.nuclph...
81,sun_c,"Sun, C.",29,; 10.1007/JHEP06(2023)176; 10.1007/JHEP10(2024...
1,park_s,"Park, S.",22,; 10.1016/j.nuclphysa.2024.122874; 10.1016/j.p...
14,drees_a,"Drees, A.",18,; 10.1007/JHEP08(2025)075; 10.1016/j.physletb....
15,esha_r,"Esha, R., Esha, Roli",16,; 10.1007/JHEP06(2023)176; 10.1007/JHEP08(2025...
17,gal_c,"Gal, C.",16,; 10.1016/j.physletb.2023.138101; 10.1103/6pmd...
8,corliss_r,"Corliss, R.",14,; 10.1007/JHEP08(2025)075; 10.1088/1742-6596/2...
10,david_g,"David, G.",14,; 10.1007/JHEP08(2025)075; 10.1103/6pmd-6dwr; ...
18,garg_p,"Garg, P.",14,; 10.1007/JHEP08(2025)075; 10.1088/1748-0221/1...
11,dehmelt_k,"Dehmelt, K.",14,; 10.1007/JHEP08(2025)075; 10.1088/1748-0221/1...
